In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
# Load the trained model 

model = load_model("model.h5")

# Load the encoded and scaler pickle files
with open("label_encoder_gender.pkl",'rb') as file:
    label_encoder_gender = pickle.load(file)

with open("onehot_encoder_geo.pkl",'rb') as file:
    oh_encoder_geo = pickle.load(file)

with open("scaler.pkl",'rb') as file:
    scaler = pickle.load(file)

In [3]:
# Example of input data

input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

approach 1:

In [4]:
label_encoder_gender.transform([input_data['Gender']])


array([1])

In [5]:
input_data['Gender'] = int(label_encoder_gender.transform([input_data['Gender']]))

C:\Users\reddy\AppData\Local\Temp\ipykernel_35788\811456339.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  input_data['Gender'] = int(label_encoder_gender.transform([input_data['Gender']]))


In [6]:
input_data

{'CreditScore': 600,
 'Geography': 'France',
 'Gender': 1,
 'Age': 40,
 'Tenure': 3,
 'Balance': 60000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 50000}

In [7]:
# pd.DataFrame(input_data.values())

In [8]:
pd.DataFrame(oh_encoder_geo.transform([[input_data['Geography']]]), columns = oh_encoder_geo.get_feature_names_out())

d:\Github\churn-prediction\my_env\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [9]:
encoded_val =oh_encoder_geo.transform([[input_data['Geography']]])
encoded_col = oh_encoder_geo.get_feature_names_out()
encoded_val
# print(encoded_val, encoded_col)

d:\Github\churn-prediction\my_env\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[1., 0., 0.]])

In [10]:
for i in range(len(encoded_col)):
    input_data[encoded_col[i]] = int(encoded_val[0][i])

In [11]:
input_data

{'CreditScore': 600,
 'Geography': 'France',
 'Gender': 1,
 'Age': 40,
 'Tenure': 3,
 'Balance': 60000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 50000,
 'Geography_France': 1,
 'Geography_Germany': 0,
 'Geography_Spain': 0}

In [12]:
input_data.pop('Geography')

'France'

In [13]:
input_data

{'CreditScore': 600,
 'Gender': 1,
 'Age': 40,
 'Tenure': 3,
 'Balance': 60000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 50000,
 'Geography_France': 1,
 'Geography_Germany': 0,
 'Geography_Spain': 0}

In [14]:
raw_input = np.array(list(input_data.values())).reshape(1,-1)
raw_input
# scaler.transform(list())

array([[  600,     1,    40,     3, 60000,     2,     1,     1, 50000,
            1,     0,     0]])

In [15]:
scaled_input = scaler.transform(raw_input)
scaled_input

d:\Github\churn-prediction\my_env\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
pred= model.predict(scaled_input)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


array([[0.06626215]], dtype=float32)

In [17]:
if float(pred.reshape(1,)[0])<0.5:
    prediction = 0
else:
    prediction = 1
    
print('Prediction for Exited: ',prediction)

Prediction for Exited:  0


approach 2:

In [18]:
input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [19]:
geo_encoded = oh_encoder_geo.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=oh_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

d:\Github\churn-prediction\my_env\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [20]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [21]:
# encoding gender col
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [22]:
# concatination of one hot encoded 
final_input_df= pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
final_input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [23]:
# scaling the data
input_scaled = scaler.transform(final_input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [24]:
# Predict churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


array([[0.06626215]], dtype=float32)

In [25]:
prediction_prob = prediction[0][0]
prediction_prob

np.float32(0.06626215)

In [26]:
if prediction_prob>0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.


In [27]:
oh_encoder_geo.categories_[0]

array(['France', 'Germany', 'Spain'], dtype=object)

In [28]:
input_data1 = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [29]:
pd.DataFrame({
    'CreditScore':[600], 
    # 'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
})

# ValueError: If using all scalar values, you must pass an index

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000
